In [447]:
#!pip install simpledorff
import simpledorff as sf
import pandas as pd
import json
import numpy as np
from glom import glom
pd.__version__
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [221]:
# load data using Python JSON module
with open('Jsonfiles/ch23_1.json','r') as f:
    data = json.loads(f.read())
# Flatten data, keep entry ID and person who completed the data
df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
    'id',
    ['annotations', 'completed_by'],
    ['annotations', 'id'],
], record_prefix = '_',
    errors = 'ignore'
                                  )

# Remove unused columns
df_base_list = df_base_list.drop(['_id', '_to_name', '_origin', '_type', 'id'], axis=1)

# Rename remaining columns
#df_base_list.rename(columns = {'_from_name':'data_type', '_value.text':'text'}, inplace = True)


# Anonymise annotator by giving them a number
df_base_list['annotations.completed_by'] = 1
df_base_list['_value.labels'] = df_base_list['_value.labels'].str[0]
df_base_list['_value.labels'] = df_base_list['_value.labels'].astype(str)

# Separate speakers and speeches
df_speakers = df_base_list[df_base_list['_value.labels'] == "Speaker"]
df_speeches = df_base_list[df_base_list['_value.labels'] != "Speaker"]

#Clean speakers dataframe by removing unused columns
series_speakers_name = df_speakers['_value.text']
series_speakers_id = df_speakers['_meta.text'].str[0]
df_speakers_final =  pd.concat([series_speakers_name, series_speakers_id], axis=1).reset_index()

# Drop all NaN values, or they'll cause issues
df_speeches = df_speeches.dropna(subset=['annotations.id'])
df_speakers_final = df_speakers_final.dropna(subset=['_meta.text'])

# Convert both columns to integer to merge
df_speeches['annotations.id'] = df_speeches['annotations.id'].astype(int)
df_speakers_final['_meta.text'] = df_speakers_final['_meta.text'].astype(int)

# Merge speakers and speeches dataframes based on matching IDs
df_final = pd.merge(df_speeches, df_speakers_final, how='inner', left_on=['annotations.id'], right_on=['_meta.text'])

# Remove speech lines and only keep those with emotions
df_final_emotions_1 = df_final[df_final['_from_name'] == 'emotion']

# TODO : Merge all 4 dataframes into one. Group_by?


# Turn emotions into numerical value
#df_final_emotions_1 = pd.get_dummies(df_final_emotions_1, columns=['_value.labels'])

#codes, unique = pd.factorize(df_final_emotions_1['_value.labels'])

# Calculate Krippendorff' alpha
#sf.calculate_krippendorffs_alpha_for_df(df_final_emotions_1)

#print(codes,unique)


In [384]:
def create_dataframe(json_path, annotator_int):
    # load data using Python JSON module
    with open(json_path,'r') as f:
        data = json.loads(f.read())
    # Flatten data, keep entry ID and person who completed the data
    df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
        'id',
        ['annotations', 'completed_by'],
        ['annotations', 'id'],
    ], record_prefix = '_',
        errors = 'ignore'
                                      )

    # load original data separately
    with open(json_path,'r') as f:
        data = json.loads(f.read())
    # Flatten data, keep entry ID and person who completed the data
    df_original_data = pd.json_normalize(data, max_level=1, meta = ['id'], record_prefix = '_',
        errors = 'ignore'
                                      )

    # Only keep relevant columns from df_original_data
    series_id = df_original_data['id']
    series_data = df_original_data['data.text']
    df_original_data = pd.concat([series_id, series_data], axis=1)


    # Add original data to dataframe
    df_base_list = pd.merge(df_base_list, df_original_data, how='inner', left_on=['id'], right_on=['id'])

    # Only keep relevant columns from df_base_list
    series_datatype = df_base_list['_from_name']
    series_start = df_base_list['_value.start']
    series_end = df_base_list['_value.end']
    series_text = df_base_list['_value.text']
    series_label = df_base_list['_value.labels']
    series_speakerid = df_base_list['_meta.text']
    series_annotator = df_base_list['annotations.completed_by']
    series_id = df_base_list['annotations.id']
    series_original = df_base_list['data.text']
    df_base_list = pd.concat([series_datatype, series_start, series_end, series_text, series_label, 
                              series_speakerid, series_annotator, series_id, series_original], axis=1)

    # Rename remaining columns
    df_base_list.columns = ['data_type', 'start', 'end', 'text', 'label', 'speaker_id', 'annotator', 'id', 'original_data']

    # Anonymise annotator by giving them a number
    df_base_list['annotator'] = annotator_int

    # Remove useless lists in labels and convert to string
    df_base_list['label'] = df_base_list['label'].str[0]
    df_base_list['label'] = df_base_list['label'].astype(str)

    # Separate speakers and speeches
    df_speakers = df_base_list[df_base_list['label'] == "Speaker"]
    df_speeches = df_base_list[df_base_list['label'] != "Speaker"]

    #Clean speakers dataframe by removing unused columns
    df_speakers = df_speakers.drop(['data_type', 'start', 'end', 'label', 'annotator', 'id', 'original_data'], axis=1)
    # Remove useless list in speaker_id
    df_speakers['speaker_id'] = df_speakers['speaker_id'].str[0]

    # Replace NaN values with -1 values, so we can convert the columns to int
    df_speeches['id'] = df_speeches['id'].fillna(-1)
    df_speakers['speaker_id'] = df_speakers['speaker_id'].fillna(-1)

    # Convert both columns to integer so we can merge later
    df_speeches['id'] = df_speeches['id'].astype(int)
    df_speakers['speaker_id'] = df_speakers['speaker_id'].astype(int)

    # Merge speakers and speeches dataframes based on matching IDs
    df_final = pd.merge(df_speeches, df_speakers, how='inner', left_on=['id'], right_on=['speaker_id'])

    # Delete unused columns and rename new ones
    df_final = df_final.drop(['speaker_id_x', 'speaker_id_y'], axis=1)
    df_final.columns = ['data_type', 'start', 'end', 'text', 'label', 'annotator', 'id', 'original_data', 'speaker']

    # Remove speech lines and only keep those with emotions
    df_final = df_final[df_final['data_type'] == 'emotion']

    return df_final

In [468]:
def merge_dataframes(json_path_list):
    dataframes = []
    # Create the dataframes from json_path_list
    for idx, file in enumerate(json_path_list):
        df = create_dataframe(file, idx)
        dataframes.append(df)

    # Concatenate all dataframes into one
    df = pd.concat(dataframes)

    # Turn emotions into numerical value
    codes, unique = pd.factorize(df['label'])
    df['label'] = codes
    
    # Create rounded length of text to use as a margin of error when grouping
    df['rounded_length'] = (df['end'] - df['start']).round(-1)
    
    
    # Group rows by original_data and length of text
    df = df.groupby(['original_data', 'rounded_length'])
    print(unique)
    
    # Calculate Krippendorff's alpha for each group TODO: FIX
    for group in df:
        kripp = sf.calculate_krippendorffs_alpha_for_df(group,experiment_col='rounded_length', annotator_col='annotator', class_col='label')
        print(kripp)

    return df

In [469]:
json_list = ['Jsonfiles/ch23_1.json', 'Jsonfiles/ch23_2.json', 'Jsonfiles/ch23_3.json', 'Jsonfiles/ch23_4.json']
test = merge_dataframes(json_list)

# display grouped information   
test.apply(lambda a: a[:])

C:\Users\gisla\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\gisla\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Index(['ColÃ¨re', 'Tristesse', 'Calme', 'Ardeur', 'Joie', 'Peur', 'Plaisir',
       'DÃ©goÃ»t'],
      dtype='object')


data_type  \
original_data                                      rounded_length                
â€” Aias , excellent pour la querelle , homme i... 270            42   emotion   
                                                                  31   emotion   
                                                                  45   emotion   
â€” Antilokhos , aucun homme n'est plus perfide... 160            70   emotion   
                                                                  51   emotion   
                                                                  37   emotion   
                                                                  76   emotion   
â€” Antilokhos , certes , Zeus et PoseidaÃ´n , ... 450            44   emotion   
                                                   730            45   emotion   
                                                   810            52   emotion   
                                                   2380           46   emotion   
                                                                  47   emotion   
                                                                  51   emotion   
                                                                  53   emotion   
â€” Antilokhos , ma colÃ¨re ne te rÃ©siste pas ... 540            28   emotion   
                                                                  22   emotion   
                                                                  30   emotion   
                                                                  31   emotion   
â€” Antilokhos , si tu veux que je prenne dans ... 220            34   emotion   
                                                                  84   emotion   
                                                                  36   emotion   
â€” Antilokhos , toi qui Ã©tais plein de sagess... 150            72   emotion   
                                                   470            27   emotion   
                                                   810            1    emotion   
                                                                  2    emotion   
                                                                  32   emotion   
                                                                  71   emotion   
â€” Antilokhos , tu mÃ¨nes tes chevaux avec imp... 50             39   emotion   
                                                   60             40   emotion   
                                                                  49   emotion   
                                                   170            41   emotion   
â€” Antilokhos , tu ne m'auras point louÃ© en v... 100            20   emotion   
                                                                  7    emotion   
                                                                  7    emotion   
                                                                  10   emotion   
â€” AtrÃ©ide , nous savons combien tu l'emporte... 80             68   emotion   
                                                   90             67   emotion   
                                                   220            14   emotion   
                                                                  1    emotion   
                                                                  1    emotion   
â€” AtrÃ©ide , Ã  qui tout le peuple Akhaien ob... 140            63   emotion   
                                                   240            52   emotion   
                                                                  54   emotion   
â€” AtrÃ©ides , et vous , princes des Akhaiens ... 130            55   emotion   
                                                   530            56   emotion   
                                                   660            58   emotion   
                                                   670            50   emotion   
â€” AtrÃ©ides , et vous , trÃ¨s braves Akhaiens... 

In [320]:

# Merge dataframes
df = dataframes[0]
    for dataframe in dataframes[1:]:
        df = df.merge(dataframe, on='original_data')

In [448]:
d = {'text': ["aaa", "bbb", "ccc", "ddd", "eee", "fff"], 'start': [0, 1, 0, 2, 1, 0], 'end': [250, 500, 501, 251, 249, 499]}
df = pd.DataFrame(data=d)
df = df.groupby(['start', 'end'])

df.apply(lambda a: a[:])

text  start  end
start end                   
0     250 0  aaa      0  250
      499 5  fff      0  499
      501 2  ccc      0  501
1     249 4  eee      1  249
      500 1  bbb      1  500
2     251 3  ddd      2  251